In [7]:
import cv2
import numpy as np
import os
from tqdm import tqdm

In [2]:
protoPath = 'deploy.prototxt'
modelPath = 'hed_pretrained_bsds.caffemodel'

In [3]:
net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

In [1]:
import gc

In [5]:
i='bag'

In [23]:
for img in tqdm(os.listdir('runs/detect/predict/crops/{}/'.format(i))):
    print(img)
    image = cv2.imread('runs/detect/predict/crops/{}/'.format(i) + img)
    # H, W = image.shape[0], image.shape[1]
    image = cv2.resize(image, (224,224))
    mean_pixel_values= np.average(image, axis = (0,1))
    blob = cv2.dnn.blobFromImage(image, scalefactor=0.77, size=(224, 224),
                                 mean=(mean_pixel_values[0], mean_pixel_values[1], mean_pixel_values[2]),
                                 # mean=(105, 117, 123),
                                 swapRB= False,
                                 crop=True)
    net.setInput(blob)
    hed = net.forward()
    hed = hed[0,0,:,:]
    hed = (255 * hed).astype("uint8")
    cv2.imwrite('hed_images/' + img, hed)
    gc.collect()

  0%|          | 0/61 [00:00<?, ?it/s]

25131.jpg


  2%|▏         | 1/61 [00:00<00:24,  2.50it/s]

25321.jpg


  3%|▎         | 2/61 [00:00<00:19,  2.99it/s]

...

25132.jpg


 31%|███       | 19/61 [00:04<00:10,  4.11it/s]

25044.jpg


 31%|███       | 19/61 [00:05<00:11,  3.78it/s]


KeyboardInterrupt: 

In [7]:
#Extracting features

In [2]:
import tensorflow as tf
import keras
import pandas
import cv2
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow import expand_dims

2023-10-27 17:35:46.666999: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-27 17:35:47.017951: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-27 17:35:51.746355: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-27 17:35:51.773753: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-27 17:35:57.503699: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=(224,224, 3), pooling='avg')

In [4]:
import pickle
ls = pickle.load(open('extracted_features_hed_images.pkl', 'rb'))

In [5]:
def preprocess(img):
    image = cv2.imread(img)
    image = cv2.resize(image,(224, 224))
    exp_image = expand_dims(image, axis=0)
    preprocessed_image = preprocess_input(exp_image)
    return preprocessed_image

In [19]:
i='skirt'
for files in tqdm(os.listdir('hed_images/{}/'.format(i))):
    image=preprocess('hed_images/{}/'.format(i) + files)
    features = vgg16.predict(image, verbose=False).flatten()
    ls.append((features, files.split('.')[0]))
    gc.collect()

100%|██████████| 2960/2960 [12:57<00:00,  3.81it/s]


In [20]:
import pickle
pickle.dump(ls, open('extracted_features_hed_images.pkl', 'wb'))

In [21]:
import joblib
joblib.dump(ls, 'extracted_features_hed_images.joblib')

['extracted_features_hed_images.joblib']

In [22]:
len(ls)

55995

In [ ]:
ls[0]